### Real or Not? NLP with Disaster Tweets

This is an introductory Kaggle competition for using NLP to predict whether a tweet is about an actual disaster or not. It is a binary classification task using the F1 score as the evaluation metric.

In [1]:
# Import required packages
import pandas as pd
import numpy as np
np.random.seed(27)
import matplotlib.pyplot as plt
plt.style.use('ggplot')
colours = plt.rcParams['axes.prop_cycle'].by_key()['color']   
%matplotlib inline
import seaborn as sns
sns.color_palette('muted')
current_palette = sns.color_palette()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import data - first two lines of code are for Kaggle submission. Second two are for local.
# train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
# test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

